# **1 : - Fake News**



# **0 :- Real News**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Printing stopwords in english
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

## **Data Pre-Processing**

In [ ]:
# Loading the Dataset to a Pandas Dataframe
#news_dataset = pd.read_csv('/content/train.csv')
import pandas as pd

# For pandas >= 1.3.0, use 'on_bad_lines' instead
news_dataset = pd.read_csv('/content/train.csv',
                          engine='python',
                          on_bad_lines='skip',  # Replaces error_bad_lines=False
                          encoding='utf-8')

print(f"Successfully loaded dataset with {len(news_dataset)} rows")
print(f"Dataset shape: {news_dataset.shape}")

Successfully loaded dataset with 1975 rows
Dataset shape: (1975, 5)


In [ ]:
news_dataset.shape

(1975, 5)

In [ ]:
# printing first five rows of  dataset
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

,0
id,0
title,51
author,209
text,4
label,0


In [ ]:
# replacing the null value with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author']+ ' ' +news_dataset['title']

In [ ]:
print(news_dataset['content'])

0       Darrell Lucus House Dem Aide: We Didn’t Even S...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
1970    Author USA – China: Who Is Responsible for Nor...
1971    David Waldstein and Ben Rothenberg U.S. Open Q...
1972    Michelle Innis Australia Is Not as Down Under ...
1973    Poke Staff 7 Tory horror film posters to send ...
1974                                           Anonymous 
Name: content, Length: 1975, dtype: object


In [ ]:
# seprating the data & label
X = news_dataset.drop(columns = 'label', axis = 1)
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

        id                                              title  \
0        0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2        2                  Why the Truth Might Get You Fired   
3        3  15 Civilians Killed In Single US Airstrike Hav...   
4        4  Iranian woman jailed for fictional unpublished...   
...    ...                                                ...   
1970  1970  USA – China: Who Is Responsible for North Kore...   
1971  1971  U.S. Open Quieted Those Calling for a Roof. No...   
1972  1972  Australia Is Not as Down Under as Everyone Thi...   
1973  1973  7 Tory horror film posters to send a chill dow...   
1974  1974                                                      

                                  author  \
0                          Darrell Lucus   
1                        Daniel J. Flynn   
2                     Consortiumnews.com   
3                        Jessica Purkiss   

In [ ]:
print(X.shape)
print(Y.shape)

(1975, 5)
(1975,)


# **## Stemming :-**
# **stemming is the process to reduce the word to its root word**

## **Example :- actor , actress , acting = act**


In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ' , content)  # removing all things except alphabtes and words
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in  stemmed_content if  not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [ ]:
print(news_dataset['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
1970    author usa china respons north korean nuclear ...
1971    david waldstein ben rothenberg u open quiet ca...
1972    michel inni australia everyon think new york time
1973    poke staff tori horror film poster send chill ...
1974                                               anonym
Name: content, Length: 1975, dtype: object


In [ ]:
# Separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michel inni australia everyon think new york time'
 'poke staff tori horror film poster send chill spine halloween' 'anonym']


In [ ]:
print(Y)

[1 0 1 ... 0 1 1]


In [ ]:
Y.shape

(1975,)

In [ ]:
# Converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [ ]:
print(X)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 19896 stored elements and shape (1975, 5509)>
  Coords	Values
  (0, 98)	0.2828424206099429
  (0, 821)	0.35525724544110054
  (0, 980)	0.25639562943636907
  (0, 1211)	0.3370138049505054
  (0, 1278)	0.2746390699269642
  (0, 1652)	0.2516550530981723
  (0, 2327)	0.23193627189537758
  (0, 2551)	0.2473666801688196
  (0, 2839)	0.29889060326268246
  (0, 2931)	0.3370138049505054
  (0, 4365)	0.26169514290578866
  (0, 5106)	0.29889060326268246
  (1, 503)	0.3013470519256806
  (1, 637)	0.1587175608416285
  (1, 738)	0.38807795762817243
  (1, 927)	0.20436653604578367
  (1, 1203)	0.2897856930445054
  (1, 1846)	0.6883566973493512
  (1, 2273)	0.19695229440722906
  (1, 5413)	0.30826548780672197
  (2, 975)	0.31623277493920554
  (2, 1036)	0.48492083192704566
  (2, 1806)	0.35635819757989223
  (2, 2006)	0.35154712234393204
  (2, 3144)	0.48492083192704566
  :	:
  (1971, 3923)	0.3552855622098234
  (1971, 4185)	0.3552855622098234
  (1971, 4197)	0.3370

# **Splitting the Dataset To training and test data**

In [ ]:
X_train, X_test , Y_train , Y_test =  train_test_split(X,Y, test_size = 0.2 , stratify = Y , random_state = 2)

# **Training the model : Logistic Regression**

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train , Y_train)

LogisticRegression()

## **Evaluation**

### **Accuracy Score**

In [ ]:
# Accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction , Y_train)

In [ ]:
print('Accuracy Score of the Training Data :' , training_data_accuracy)

Accuracy Score of the Training Data : 0.9727848101265822


In [ ]:
# Accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction , Y_test)

In [ ]:
print('Accuracy Score of the Training Data :' , test_data_accuracy)

Accuracy Score of the Training Data : 0.9265822784810127


# **Making Predictive System **

In [ ]:
X_new = X_test[0]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print('The News is Real :')
else:
  print('The News Is Fake')

[1]
The News Is Fake


In [ ]:
print(Y_test[0])

1


In [ ]:
X_new = X_test[5]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print('The News is Real :')
else:
  print('The News Is Fake')

[0]
The News is Real :


In [ ]:
print(Y_test[5])

0
